# Selective Intersection Over Union
Ovdje smo koristili selective search u kombinaciji sa Intersection Over Union kako bismo pomoću selective searcha i već određenih groun truth lokacija za tražene logoe stvorili skupove slika na kojima je selective search dao dobar prijedlog traženog logoa. Također smo i sve što nije prema IoU metodi bilo dobro, spremili u u Other/Background klasu kako bi imali i ono što ne znamo što je. Ono što je SS ispravno predložio, spremali smo u formatu 60x40

In [ ]:
import sys
import cv2
import time
import os
import uuid
import json
import datetime

MARKEDLOGOS = '/home/bosnmar/MachineLearning/Production/MarkedLogos/'
SELECTIVELOGO = "/home/bosnmar/MachineLearning/Production/SelectiveLogo/"
OTHER = "/home/bosnmar/MachineLearning/Production/TrainingSet/ResAugTrLogo/Other/"



def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    if (xB-xA) < 0 or (yB - yA) < 0:
        return 0
    # compute the area of intersection rectangle
    interArea = float((xB - xA + 1) * (yB - yA + 1))
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = float((boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1))
    boxBArea = float((boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1))
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = float(interArea / float(boxAArea + boxBArea - interArea))

    #print (iou)
 
    # return the intersection over union value
    return iou


def selectivesearch(img_path, logopoz, picclass):
    # speed-up using multithreads

    im = cv2.imread(MARKEDLOGOS + picclass + "/" +img_path)

    
    # read image
    Height=400
    Width=220

    my_counter = 0

    try:    
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(im)

        ss.switchToSelectiveSearchFast()
        # run selective search segmentation on input image
            
        #funkcija vrati array sa koordinatama rectangleova [x1 y1 w h] w->sirina h->visina dakle x1 y1 x1+w y1+h so koord od recta
        rects = ss.process()
        threshold_rects=[]

        #cuva samo rectangle takve da je povrsina >= povrsinaslike/100
        for i,rect in enumerate(rects):
            x, y, w, h = rect
            if (w*h >= float(Height*Width)/100): 
                threshold_rects.append(rect)

        broj_rectangle_u_klasi=0 
        imOut = im.copy()
        for i,rect in enumerate(threshold_rects):
            my_counter = my_counter + 1
            x ,y, w, h = rect
            #funkciji bb_intersection_over_union moras dati gornji lijevi i donji desni pa to izvuci iz Logopoza 
            iou = bb_intersection_over_union([x,y,x+w,y+h] , [logopoz[0],logopoz[1],logopoz[0]+logopoz[2],logopoz[1]+logopoz[3]])
            #print(iou)
            if (float(iou) > float(0.4)):
                 #prebaci u crno bijelu boju
                crop_log = imOut[y:y+h, x:x+w]
                crop_log = cv2.cvtColor(crop_log, cv2.COLOR_BGR2GRAY)
                cv2.imwrite( SELECTIVELOGO + picclass + "/" + str(broj_rectangle_u_klasi) + "_" + img_path, crop_log)
                broj_rectangle_u_klasi= broj_rectangle_u_klasi+1
            else:
                other = imOut[y:y+h, x:x+w]
                other = cv2.resize(other, (60, 40))
                other = cv2.cvtColor(other, cv2.COLOR_BGR2GRAY)
                other_name = str(uuid.uuid4()) + ".jpg"
                cv2.imwrite(OTHER + other_name, other)

            
        Omjer=broj_rectangle_u_klasi/len(threshold_rects) 
        return Omjer
    except:
        print ("Class: {}; Picture: {}".format(piccl, img_path))
        return 0
    
"""
def create_train_skup:
    arrayomjera=[0]*25
    arraycounter=[0]*25
    for petlja po folderima ili cemu vec imas spremljene slike (u boji) i pozicije pripadnih loga:
        #klasa slike neka bude neki broj od 0 do 24 ili 1 do 25 kako god
        omjer=selectivesearch(slika,pozicijaloga,klasaslike)
        arrayomjera[klasaslike]=arrayomjera[klasaslike]+omjer
        arraycounter[klasaslike]=arraycounter[klasaslike]+1
        
    for i in range(0,25):
        arrayomjer[i]=arrayomjer[i]/arraycounter[i]
        
    return arrayomjer #u njemu sada imamo sve omjere i to zapamti i cuvaj ko oci u glavi 

def getMyLogo(imgClass, img):
"""

def getLogo(imgClass, img, data):
    for item in data[imgClass]:
        temp = item[0]
        temp = temp[temp.index("_")+1:]
        if str(temp) == str(img):
            #print ("Lookig for: {}".format(img))
            #print ("Found:      {}".format(temp))
            return  item[1:]


def loadJson():
    with open('logo_coord_backup.json') as f:
        return json.load(f)
    
    

if __name__ == "__main__":

    cv2.setUseOptimized(True)
    cv2.setNumThreads(20)

    start = datetime.datetime.now()

    my_classes = [piccl for piccl in os.listdir(MARKEDLOGOS)]
    my_locations = loadJson()

    i = 0

    for piccl in my_classes:
        if not os.path.exists(SELECTIVELOGO + piccl):
            os.makedirs(SELECTIVELOGO + piccl + "/")

    if not os.path.exists(OTHER):
        os.makedirs(OTHER)

    om_co = {}
    omjerClass = {}
    for piccl in my_classes:
        om_co.update({piccl:[0, 0]})
        omjerClass.update({piccl : 0})

    print (om_co)
    for piccl in my_classes:
        images = [img for img in os.listdir(MARKEDLOGOS + piccl + "/")]
        print("Starting with class: {}".format(piccl))
        print("Start time {}".format(datetime.datetime.now()))

        for img in images:
            i = i + 1
            if (i%4 == 0):
                logopoz = getLogo(piccl, img, my_locations)
                omjer = selectivesearch(img, logopoz, piccl)
                #print ("IMG: {}: Omjer: {}".format(img,omjer))
                om_co[piccl][0] = om_co[piccl][0] + omjer
                om_co[piccl][1] = om_co[piccl][1] + 1
            if (i % 200 == 0):
                print("Went through {} images".format(i))
        
        print("Ended with class: {}".format(piccl))
        print("End time {}\n".format(datetime.datetime.now()))

    
    #print (om_co)
        
    for key in omjerClass.keys():
        if om_co[key][1] > 0:
            omjerClass[key] = float(om_co[key][0]/om_co[key][1])

    print(omjerClass)
    """
    with open('omjerclass.json', 'w') as outfile:
        json.dump(omjerClass, outfile)"""    

    print("Started at: {}".format(start))
    print("Ended at: {}".format(datetime.datetime.now()))



    